#### Core Premise
We currently have a model that uses 8 months of training data to generate predictions. We would like to experiment to see if a version of the model which trains on 4 months of training data to predict conversion in the following 2 month period could be viable, so that the lead time to use is reduced.

In production, this could be used in such a way that the list of users predicted to convert within those two months can be placed on a priority list for a limited discount-promotions budget* Meanwhile, the data for the next round of predictions is being collected. 

For the purpose of the experiment, the available data will be divided into 6-month blocks, with the first 4 months of each block constituting the data collection period and the last two months constitutiing the performance period in which the target is whether or not a conversion took place. Subsets of these collection period + performance period pairs will be used for the training set, validation set, and test set respectively.

**Exploratory data analysis of relationship between adwords promotions & conversion was low, so a different type of nudge is suggested here*

### Preprocessing with Cutoff Date

In [1]:
import pandas as pd

In [2]:
import gc

In [3]:
gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")

/var/folders/z1/1zds15w97_g7_fhs4_83dc3m0000gn/T/ipykernel_63499/3136096564.py:1: DtypeWarning: Columns (2,54) have mixed types. Specify dtype option on import or set low_memory=False.
  gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")


In [4]:
gstore['date'] = pd.to_datetime(gstore['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

In [6]:

print("Earliest Date:", (gstore['date']).min())
print("Latest Date:", (gstore['date']).max())

Earliest Date: 2016-08-01
Latest Date: 2017-08-01


6-month split is end of Feb 2017. First set will be beginning of August to end of Feb, second set will be beginning of March to beginning of August

In [5]:
cutoff_date = pd.to_datetime('2017-02-28')

In [6]:

gstore['date'] = pd.to_datetime(gstore['date'])
print(gstore['date'].dtype)

datetime64[ns]


In [7]:
df1 = gstore.loc[gstore['date'] <= cutoff_date]
df2 = gstore.loc[gstore['date'] > cutoff_date]

In [11]:
print(df1.shape[0] + df2.shape[0])

903653


In [8]:
# data collection cutoff dates (4 months in)
df1cutoff = pd.to_datetime('2016-11-30') 
df2cutoff = pd.to_datetime('2017-06-30')

In [9]:
# set aside collection period and performance period data
df1p = df1.loc[df1['date'] > df1cutoff]  # performance
df1c = df1.loc[df1['date'] <= df1cutoff] # collection

In [10]:
df1trainids = df1c['fullVisitorId'].unique().tolist() # getting list of IDs for which we will check conversion in performance period

In [11]:
df1train = pd.DataFrame(df1trainids, columns=['fullVisitorId']) # beginnning construction of training dataframe

In [19]:
gstore.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'continent', 'subContinent', 'country', 'region', 'metro', 'city',
       'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation',
       'browser', 'browserVersion', 'browserSize', 'operatingSystem',
       'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding',
       'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo',
       'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors',
       'screenResolution', 'deviceCategory', 'visits', 'hits', 'pageviews',
       'bounces', 'newVisits', 'transactionRevenue', 'campaign', 'source',
       'medium', 'keyword', 'adwordsClickInfo.criteriaParameters',
       'isTrueDirect', 'referralPath', 'adwordsClickInfo.page',
       'adwordsClickInfo.slot', 'adwordsClickInfo.gclId',
       'adwordsClickInfo.adNetworkType', 'adwordsClickInfo.isVideoAd',
       '

In [12]:
# identified as variables which would not aid analysis in earlier EDA steps
to_drop = ['socialEngagementType', 'networkDomain', 'latitude', 'longitude', 'networkLocation',
       'browser', 'browserVersion', 'browserSize', 'operatingSystem',
       'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding',
       'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo',
       'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors',
       'screenResolution', 'bounces', 'keyword', 'referralPath','campaignCode']

In [13]:
# to make parsing through df1c a bit more efficient
for col in to_drop:
    if col in df1c.columns:
        df1c.drop(col, axis=1, inplace = True)

/var/folders/z1/1zds15w97_g7_fhs4_83dc3m0000gn/T/ipykernel_63499/2759826767.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1c.drop(col, axis=1, inplace = True)
/var/folders/z1/1zds15w97_g7_fhs4_83dc3m0000gn/T/ipykernel_63499/2759826767.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1c.drop(col, axis=1, inplace = True)
/var/folders/z1/1zds15w97_g7_fhs4_83dc3m0000gn/T/ipykernel_63499/2759826767.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [14]:
# create label for conversion in performance period

# Step 1: Create a Boolean Series where True represents a non-null transactionRevenue
converted_visitors = df1p['transactionRevenue'].notnull()

# Step 2: Group by 'fullVisitorId' in df1p and check if any transactionRevenue entries are not null
conversion_status = df1p[converted_visitors].groupby('fullVisitorId').size() > 0

# Step 3: Map the conversion status back to df1c to create the 'targetConversion' column
df1train['targetConversion'] = df1train['fullVisitorId'].map(conversion_status).fillna(0).astype(int)


In [23]:
# turning it into a function to reuse
def conv_label(performance_df, groupby_col, result_df):
    """
    Creates a 'targetConversion' column in the result DataFrame based on conversion data from the performance DataFrame.
    
    Parameters:
        performance_df (pd.DataFrame): DataFrame containing performance data, including 'transactionRevenue'.
        groupby_col (str): Column name to group by, typically 'fullVisitorId'.
        result_df (pd.DataFrame): DataFrame to which the 'targetConversion' results will be added.
        
    Returns:
        pd.DataFrame: The modified result DataFrame with a new 'targetConversion' column.
    """
    # Step 1: Create a Boolean Series where True represents a non-null transactionRevenue
    converted_visitors = performance_df['transactionRevenue'].notnull()

    # Step 2: Group by the specified column in performance_df and check if any transactionRevenue entries are not null
    conversion_status = performance_df[converted_visitors].groupby(groupby_col).size() > 0

    # Step 3: Map the conversion status back to the result DataFrame to create the 'targetConversion' column
    result_df['targetConversion'] = result_df[groupby_col].map(conversion_status).fillna(0).astype(int)

    return result_df

In [27]:
sum(df1train['targetConversion'])

409

In [15]:
# creating column for if user visited for first time in collection period
firstime_visitors = df1c['newVisits'].notnull()
firstime_status = df1c[firstime_visitors].groupby('fullVisitorId').size() > 0
df1train['newVisits'] = df1train['fullVisitorId'].map(firstime_status).fillna(0).astype(int)

In [24]:
df1c.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'continent', 'subContinent', 'country',
       'region', 'metro', 'city', 'cityId', 'deviceCategory', 'visits', 'hits',
       'pageviews', 'newVisits', 'transactionRevenue', 'campaign', 'source',
       'medium', 'adwordsClickInfo.criteriaParameters', 'isTrueDirect',
       'adwordsClickInfo.page', 'adwordsClickInfo.slot',
       'adwordsClickInfo.gclId', 'adwordsClickInfo.adNetworkType',
       'adwordsClickInfo.isVideoAd', 'adContent'],
      dtype='object')

In [29]:
columns_to_encode = ['channelGrouping', 'deviceCategory', 'source']
df1c = pd.get_dummies(df1c, columns=columns_to_encode)

In [30]:
df1c.columns

Index(['date', 'fullVisitorId', 'sessionId', 'visitId', 'visitNumber',
       'visitStartTime', 'continent', 'subContinent', 'country', 'region',
       ...
       'source_wanelo.com', 'source_wap.sogou.com', 'source_web.facebook.com',
       'source_web.mail.comcast.net', 'source_web.telegram.org',
       'source_wheretoget.it', 'source_xbidprodmirror.corp.google.com',
       'source_yahoo', 'source_yandex', 'source_youtube.com'],
      dtype='object', length=263)

In [32]:
agg_dict = {
    'country': 'first',
    'continent': 'first',
    'subContinent': 'first',
    'transactionRevenue': 'sum',
    'pageviews': 'sum',
    'isTrueDirect': 'sum'

}

In [33]:
dummy_columns = [col for col in df1c.columns if '_' in col]

In [34]:
for dummy_col in dummy_columns:
    agg_dict[dummy_col] = 'sum'

In [35]:
# making sure columns match
incorrect_cols = [key for key in agg_dict if key not in df1c.columns]
print(incorrect_cols)


[]


In [39]:
df1c['isTrueDirect'] = df1c['isTrueDirect'].map({'true': 1}).fillna(0).astype(int)

In [40]:
# fixing an error
isTrueDirect_column = df1.loc[df1['date'] <= df1cutoff, 'isTrueDirect']

df1c['isTrueDirect'] = isTrueDirect_column

In [42]:
df1c['isTrueDirect'].value_counts()

True    87254
Name: isTrueDirect, dtype: int64

In [43]:
df1c['isTrueDirect'] = df1c['isTrueDirect'].fillna(False).astype(int)

In [44]:
df1c['isTrueDirect'].value_counts()

0    270015
1     87254
Name: isTrueDirect, dtype: int64

In [45]:
useragg = df1c.groupby('fullVisitorId').agg(agg_dict)

In [ ]:
# might need to do outside: 
# count of how many times each visitor appears
# count of non-null in transrev
# visits where adwordsClickInfo.gclId is populated 
# visits where campaign is populated

In [46]:
aggregations = {
    # Count how many times each visitor appears
    'fullVisitorId': 'count',
    # Count non-null records for transactionRevenue
    'transactionRevenue': lambda x: x.notnull().sum(),
    # Count rows where adwordsClickInfo.gclId is populated
    'adwordsClickInfo.gclId': lambda x: x.notnull().sum()
}

In [47]:
visitor_counts = df2.groupby('fullVisitorId').agg(aggregations)

In [49]:
visitor_counts.columns = ['visits', 'conversions', 'adVisits']

In [51]:
visitor_counts.head()

,visits,conversions,adVisits
fullVisitorId,,,
26722803385797,1,0,0
39460501403861,1,0,0
48421062322244,1,0,0
58970809397690,1,0,0
61609983499251,1,0,0


In [52]:
visitor_counts.reset_index(inplace=True)

In [53]:
visitor_counts.head()

,fullVisitorId,visits,conversions,adVisits
0,26722803385797,1,0,0
1,39460501403861,1,0,0
2,48421062322244,1,0,0
3,58970809397690,1,0,0
4,61609983499251,1,0,0


In [59]:
visitor_counts['fullVisitorId'].dtype

dtype('O')

In [54]:
df1train = df1train.merge(visitor_counts, on='fullVisitorId', how='left')

object object


In [55]:
df1train.head()

,fullVisitorId,targetConversion,newVisits,visits_x,conversions_x,adVisits_x,visits_y,conversions_y,adVisits_y
0,1131660440785968503,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,377306020877927890,0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,3895546263509774583,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,4763447161404445595,0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,27294437909732085,0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
useragg.head()

NameError: name 'useragg' is not defined

In [31]:
# quick test of type of object agg returns
# aggtest = df1c.groupby('fullVisitorId').agg(agg_dict) # worked 
# aggtest = df1c.groupby('fullVisitorId').agg({'transactionRevenue': ['sum', ('conversions', lambda x: x.notnull().sum())]}) # did not work

In [ ]:
# try doing it this way: create all the session level flags. then do the "first" based columns, then the count of visitorid columns, then all the group by aggregations

In [26]:
# Step 1: Create the OneHotEncoder instance
encoder = OneHotEncoder(sparse=False)

# Step 2: Fit and transform the data
channel_encoded = encoder.fit_transform(df1c[['channelGrouping']])

# Step 3: Convert the numpy array back to a DataFrame
channel_encoded_df = pd.DataFrame(channel_encoded, columns=encoder.get_feature_names_out(['channelGrouping']))

# Step 4: Concatenate the new dataframe with the original dataframe minus the original 'channelGrouping' column
df1c = pd.concat([df1c.drop('channelGrouping', axis=1), channel_encoded_df], axis=1)

/Users/aoluwolerotimi/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [28]:
# df1c.columns

In [29]:
cg = ['channelGrouping_(Other)', 'channelGrouping_Affiliates',
       'channelGrouping_Direct', 'channelGrouping_Display',
       'channelGrouping_Organic Search', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social']

In [32]:
df1train = df1train[['fullVisitorId']]

In [30]:
# for column in cg:
#     df1train[column] = pd.NA

In [33]:
# Step 1: Group by 'fullVisitorId' in df1c and sum the columns in cg
grouped = df1c.groupby('fullVisitorId')[cg].sum()

# Step 2: Merge this grouped data back into df1train
df1train = df1train.merge(grouped, on='fullVisitorId', how='left')

              fullVisitorId  channelGrouping_(Other)  \
0       1131660440785968503                      0.0   
1        377306020877927890                      0.0   
2       3895546263509774583                      0.0   
3       4763447161404445595                      0.0   
4         27294437909732085                      0.0   
...                     ...                      ...   
299069  3118235170138318723                      0.0   
299070  2900750270670203622                      0.0   
299071  4821618326715417613                      0.0   
299072  2720938310295703834                      0.0   
299073   593617143165950851                      0.0   

        channelGrouping_Affiliates  channelGrouping_Direct  \
0                              0.0                     0.0   
1                              0.0                     0.0   
2                              0.0                     0.0   
3                              0.0                     0.0   
4                

In [34]:
# Step 1: Count occurrences of each 'fullVisitorId' in df1c
visitor_counts = df1c['fullVisitorId'].value_counts().reset_index()
visitor_counts.columns = ['fullVisitorId', 'totalVisits']

# Step 2: Merge this count DataFrame back into df1train
df1train = df1train.merge(visitor_counts, on='fullVisitorId', how='left')

In [35]:
# continue later
# df1c.to_csv('df1c.csv', index=True)
# df1train.to_csv('df1train.csv', index=True)

* Channel grouping dummies then sum --> DONE
* Count appearances of vistor id for total visits in period
* First Continent, Suboncontinent, Country appearing
* Device category dummies then summed
* Sum of hits
* Sum of pageviews
* Flag for if new visitor in the period
* Sum of transaction value in the period (might need to set nan to 0 first)
* Source dummies then summed then PCA
* Sum isTrueDirect
* adWordsgclID sum not null 
* Sum campaign not null (set not set to null)